Access to a file and creation of embedding in a dataframe

In [27]:
file = open("stort.txt",'r')
text = file.readlines()
file.close

UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 2976: character maps to <undefined>

In [ ]:
def text_split(text):
    word=''
    sentence=''
    paragraph=[]
    for row in text:
        for char in row:
            if char !=' ':
                word=word+char
                #print(word)
            else:
                sentence=sentence+word
                #print(sentence)
                word=' '
            if len(sentence.split())==300:
                #print(sentence)
                paragraph.append(sentence)
                sentence=' '
    paragraph.append(sentence+word) 
    print(paragraph[-1])   
    return paragraph


In [ ]:
paragraph = text_split(text)

  cestino da picnic. Il cielo si era tinto di arancione e viola, e il vento portava con sÃ© il profumo dei fiori estivi. Alessia guardava l'orizzonte con una luce negli occhi che sembrava raccontare mille storie. "Sai," disse all'improvviso, "a volte penso che siamo qui per vivere ogni singolo momento con tutta l'anima. Non importa quanto grande o piccolo sia."
Fu allora che capii quanto fosse importante per me. Alessia aveva la capacitÃ  di vedere il mondo con occhi diversi, di trovare poesia anche nei momenti piÃ¹ ordinari. Con lei, ogni giorno era un regalo, una nuova pagina del nostro racconto.
Il tempo passava, e il nostro legame diventava sempre piÃ¹ forte. Abbiamo affrontato insieme le difficoltÃ , riso fino alle lacrime, e costruito ricordi che porteremo sempre nel cuore. Alessia non era solo la mia compagna; era la mia musa, la mia forza, e la mia migliore amica. Con lei, ho imparato che l'amore non Ã¨ solo un sentimento, ma una scelta quotidiana di essere presenti, di condivi

Create text embeddings

In [ ]:
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def embedding_text(text):
    encoding = tokenizer.batch_encode_plus([text],
                padding='max_length',
                max_length=256,
                truncation=True,
                return_tensors='pt',
                add_special_tokens=True)
    input_ids = encoding['input_ids']
    return input_ids

for i in range(len(paragraph)):
    paragraph[i]=embedding_text(paragraph[i])
    print(paragraph[i])



tensor([[  101,  1039,  1005,  3690, 14477, 26089, 14477, 17768, 10936,  4143,
          4487,  2053,  4168, 15669, 18719,  2050,  1010,  6335, 12731,  2072,
          2061, 18752,  6499, 13642,  3567,  2474,  6178,  6305,  6590,  4487,
          5665, 12717, 26148,  6335, 12256,  3527,  6970,  2080,  1012, 13642,
          3567, 11192,  4048,  4880,  6894,  3459,  7088, 18178,  4907,  4143,
          6212,  2080,  9530,  6335, 18834,  2080,  1041,  1051, 25955,  3609,
          2053, 14693,  6030, 18178,  7367, 14905,  2527,  6212,  2080,  9530,
          6528,  7869, 10722,  4674,  3393, 15544, 13102, 14122,  2063,  2035,
          2063, 14383,  5685,  2063, 24624, 27904, 11268, 15422,  2063,  8611,
         19300,  1012, 15669, 18719,  2050,  2512,  3690,  3948, 12101,  1025,
         13642,  3567,  4895, 12731,  5686,  8991, 15286,  1041, 14477,  2273,
          2618,  7987,  9386, 10111,  1010,  7367,  8737,  2890,  4013, 12380,
          1037,  8292, 18992,  2890,  2474,  9852, 2

In [ ]:
user_question = 'Che era in grado di vedere alessia'
user_emb = embedding_text(user_question)
print(user_emb.size())
print(paragraph[0].size())

max_match=0
match_paragraph=[]
for ele in paragraph:
    print(cosine_similarity(user_emb,ele))
    if cosine_similarity(user_emb,ele) > max_match:
        
        max_match = cosine_similarity(user_emb,ele)
        match_paragraph=[ele,max_match]


print(tokenizer.decode(match_paragraph[0][0], skip_special_tokens=True))
print(match_paragraph[1])

torch.Size([1, 256])
torch.Size([1, 256])
[[0.1211085]]
[[0.10970345]]
c'era una volta una ragazza di nome alessia, il cui sorriso aveva la capacita di illuminare il mondo intero. aveva lunghi capelli castani che danzavano con il vento e occhi color nocciola che sembravano contenere tutte le risposte alle domande pia¹ profonde della vita. alessia non era solo bella ; aveva un cuore gentile e una mente brillante, sempre pronta a cercare la bellezza nei piccoli dettagli della vita. era un pomeriggio d'estate quando la incontrai per la prima volta. il sole splendeva alto nel cielo, e io ero seduto su una panchina del parco con un libro che, ironia della sorte, non riuscivo a leggere. lei passa² di la¬, con un cagnolino al guinzaglio, e per un istante il tempo sembra² fermarsi. il suo sorriso era un raggio di luce che mi
[[0.1211085]]
